In [5]:
from google.colab import drive
#drive.mount('/content/drive')
%cd /content/drive/MyDrive/ColabNotebooks/
%pwd
%ls -d */



/content/drive/MyDrive/ColabNotebooks
 JobSeekerAgent_langchain//  'Sample Files/'/
/content/drive/MyDrive/ColabNotebooks/JobSeekerAgent_langchain


In [9]:
!git clone https://github.com/benzp/JobSeekerAgent_langchain.git

%pwd
%cd JobSeekerAgent_langchain

fatal: destination path 'JobSeekerAgent_langchain' already exists and is not an empty directory.
/content/drive/MyDrive/ColabNotebooks/JobSeekerAgent_langchain


In [11]:
!pip install streamlit pyngrok langchain langchain-openai langchain-community langchain-tavily python-docx


In [41]:
%pwd
%ls -la
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')


total 55
-rw------- 1 root root  1454 Dec 11 13:47  app.py
drwx------ 8 root root  4096 Dec 11 12:32  .git/
-rw------- 1 root root 27649 Dec 11 13:52  JobSeekerAgent.ipynb
-rw------- 1 root root  2788 Dec 11 13:46  jobseeker_utils.py
drwx------ 2 root root  4096 Dec 11 13:47  __pycache__/
-rw------- 1 root root    73 Dec 11 12:32  README.md
-rw------- 1 root root 14419 Dec 11 12:32 'Sample Benz Resume.docx'


In [51]:
%%writefile app.py
# paste the entire streamlit UI code here
from tempfile import NamedTemporaryFile

import os

import streamlit as st
import jobseeker_utils

# =========================
# Streamlit Page Config
# =========================

st.set_page_config(
    page_title="AI Job Assistant",
    layout="wide",
)

st.title("AI Job Search & Resume Assistant")

st.write(
    """
Provide your **current skills/experience** (via resume upload). And provide your ***aspirational role*** (via text input).
"""
)

st.subheader("2️⃣ Current Skills & Experience")
resume = st.file_uploader("Upload your current resume (.pdf, .docx, .txt):")

st.subheader("3️⃣ Aspirational Role")
aspirational_role = st.text_area("Enter the aspirational role(s) you want to apply for:")

submitted = st.button("Search Ideal Jobs")

if submitted:
    # Show what the user provided
    st.write("Submitted!")
    if aspirational_role:
        st.write("Aspirational role:", aspirational_role)
    else:
        st.write("No aspirational role entered.")
    if resume is not None:
        st.write("Uploaded Resume File name:", resume.name)
        resume_contents = jobseeker_utils.extract_text_from_file(resume)
        st.write("Resume Contents:", resume_contents)
        st.subheader("Matched Search Job Results")

        final_output = jobseeker_utils.TopMatchingJobs(aspirational_role, resume_contents)
        st.write(final_output)
    else:
        st.write("No resume uploaded.")



Overwriting app.py


In [23]:
!pkill -f "streamlit run app.py" || echo "no old streamlit"

# Start Streamlit fully detached
!nohup streamlit run app.py --server.port 8501 --server.headless true > /tmp/streamlit.log 2>&1 &


^C


In [24]:
from pyngrok import ngrok

ngrok.kill()

ngrok.set_auth_token('36f8VolnFP2Ilo6ufMvTfNijTx8_2GfmfdoiDoAZkK4KsuLVA')

public_url = ngrok.connect('8501')
public_url

<NgrokTunnel: "https://luciana-unsurprised-kristi.ngrok-free.dev" -> "http://localhost:8501">

In [47]:
%%writefile jobseeker_utils.py
import os
import json
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langchain.tools import tool        # decorator to wrap Python functions as tools
from langchain.agents import create_agent
from tempfile import NamedTemporaryFile
from docx import Document



def TopMatchingJobs(aspirations: str, current_skills: str) -> str:

    tavily_search = TavilySearch(
        max_results=5,
        topic="general",
    )

    # --- Set up LLM (OpenAI chat model) ---
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.2)
    tools = [tavily_search]


    # --- Build a prompt / system instruction for the agent ---
    system_prompt = f"""
    You are an AI job-search assistant. Use the tool 'tavily_search' to search LinkedIn, Naukri and Monster job listings.
    Given the user's aspirational role text, create 1-2 suitable site:linkedin.com/jobs search queries,
    call the tavily_search tool with those queries across LinkedIn, Naukri and Monster, then produce a numbered list of the top 5 matching jobs with a higher matching score.
    Higher Matching Score Calculation: Compare against the required job description and skillsets required with the job listings against the current skills provided by the user. More the matching score, more the relevance.
    Return ONLY the numbered list (1..5) with Job Title, Company, Location, URL, Matching_Score, Matching rationale, One-liner summary of the key essence of the role requirement.
    While providing the output sort them according to the matching score in descending order.
    """

    # --- Create the agent ---
    # create_agent signature may vary; many versions accept llm=llm, tools=[...], system_prompt=...
    agent_executor = create_agent(model=llm, tools=tools, system_prompt=system_prompt)


    result = agent_executor.invoke({
        "messages": [
            {"role": "user", "content": f"User's Aspirational Role: {aspirations}; User's Current Experiences: {current_skills}"}
        ]
    })
    return result["messages"][-1].content


def extract_text_from_file(uploaded_file) -> str:
    if uploaded_file is None:
        return ""

    name = uploaded_file.name.lower()
    file_bytes = uploaded_file.read()

    if name.endswith(".docx"):
        with NamedTemporaryFile(delete=False, suffix=".docx") as tmp:
            tmp.write(file_bytes)
            tmp_path = tmp.name
        doc = Document(tmp_path)
        text = "\n".join(p.text for p in doc.paragraphs)
        os.remove(tmp_path)
        return text
    else:
        raise ValueError("Unsupported file type. Use .docx only")




Overwriting jobseeker_utils.py


In [5]:
aspirational_role = "Tech Co-Founder in Bangalore"
current_skills = """Certified AI & Data Science Leader with over 20 years of industry experience and a strong passion for teaching and
mentoring professionals in the domains of Generative AI, Agentic AI, Machine Learning and Data Science. Recognized for designing enterprise-grade
Gen AI, Agentic AI, AI/ML solutions and leading global analytics teams across compliance, risk, and data innovation domain.Currently working as
Risk Analytics Manager / Transformation Lead at Honeywell, architecting Gen AI-driven compliance frameworks. Adept at conducting workshops and
building AI literacy among business and technical teams."""

result = TopMatchingJobs(aspirational_role, current_skills)
print(result)

1. Job Title: AI & ML Engineering Lead / Co-Founder  
   Company: Datafoundry  
   Location: Bangalore, Hyderabad & Vizag  
   URL: https://am.linkedin.com/company/datafoundryai  
   Matching_Score: 0.995  
   Matching rationale: Strong match with AI, ML, compliance, risk analytics, and leadership in Bangalore; experience with scalable AI systems and compliance frameworks aligns well.  
   One-liner summary: Lead AI-driven engineering and product development focusing on compliance and risk analytics in a high-growth AI startup environment.

2. Job Title: CEO & Co-Founder  
   Company: rampp.ai  
   Location: Bangalore, Karnataka, India  
   URL: https://in.linkedin.com/in/ajay-agrawal-97573a3  
   Matching_Score: 0.630  
   Matching rationale: Relevant entrepreneurial role in AI/GenAI domain in Bangalore; leadership and startup experience align with aspirational role.  
   One-liner summary: Lead and scale an AI startup focused on enterprise transformation and generative AI solutions.


In [57]:
## Check in the files in the GitHub Repository
%pwd

!git config --global user.email "benzp2@gmail.com"
!git config --global user.name "benzp"
!git status
#!git add jobseeker_utils.py app.py JobSeekerAgent.ipynb

!git commit -m "Added new files into the repository"


On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   JobSeekerAgent.ipynb
	new file:   app.py
	new file:   jobseeker_utils.py

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   JobSeekerAgent.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	__pycache__/

[main 6601414] Added new files into the repository
 3 files changed, 117 insertions(+)
 create mode 100644 JobSeekerAgent.ipynb
 create mode 100644 app.py
 create mode 100644 jobseeker_utils.py


In [ ]:
import langchain, sys
print("langchain:", langchain.__version__)
import pkgutil
print("langchain.agents available members:")
print([m.name for m in pkgutil.iter_modules(langchain.agents.__path__)])

langchain: 1.1.2
langchain.agents available members:
['factory', 'middleware', 'structured_output']
